In [19]:
!pip install torch transformers einops accelerate matplotlib

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 890.1/890.1 MB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 74.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 MB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 83.8 MB/s eta 0:00:00
  Attempting uninstall: torch
    Found existing installation: torch 2.0.1+cu118
    Uninstalling torch-2.0.1+cu118:
      Successfully uninstalled torch-2.0.1+cu118
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchaudio 2.0.2+cu118 requires torch==2.0.1, but you have torch 1.13.0 which is incompatible.
torchdata 0.6.1 requires torch==2.0.1, but you have torch 1.13.0 which is incompatible.
torchtext 0.15.2 requires torch==2.0.1, but you have torch 1.13.0 which

In [20]:
import einops, accelerate

In [21]:
mport os
import time
import torch
import matplotlib.pyplot as plt
from typing import Tuple, List
from transformers import AutoModelForCausalLM, AutoTokenizer

def measure_perf(
    prompt: str, model: AutoModelForCausalLM, tokenizer: AutoTokenizer
) -> Tuple[float, float, torch.Tensor]:
    """
    Measures memory consumption and inference execution time for a given model and prompt.

    Args:
    prompt: Text to be used as input for the model.
    model: Pretrained model used for inference.
    tokenizer: Pretrained tokenizer used to encode the prompt.

    Returns:
    Peak memory consumption in GB, execution time in seconds, and output tensor from the model.
    """
    torch.cuda.empty_cache()
    torch.cuda.reset_peak_memory_stats()

    start_time = time.time()

    input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to("cuda")
    outputs = model.generate(input_ids, max_length=100)

    end_time = time.time()

    peak_mem = torch.cuda.max_memory_allocated()
    peak_mem_consumption = peak_mem / 1e9  # convert bytes to GB

    exec_time = end_time - start_time

    return peak_mem_consumption, exec_time, outputs


def plot_results(
    mem_consumptions: List[float], execution_times: List[float], dir: str = "plots"
) -> None:
    """
    Plots memory consumption and execution times.

    Args:
    mem_consumptions: List of memory consumption data in GB.
    execution_times: List of execution time data.
    dir: Destination dir for the plot.
    """
    os.makedirs(dir, exist_ok=True)

    fig, ax1 = plt.subplots()

    color = "tab:red"
    ax1.set_xlabel("Runs")
    ax1.set_ylabel("GPU Memory Consumption (GB)", color=color)
    ax1.plot(mem_consumptions, color=color)
    ax1.tick_params(axis="y", labelcolor=color)
    ax1.yaxis.get_major_formatter().set_useOffset(False)

    ax2 = ax1.twinx()
    color = "tab:blue"
    ax2.set_ylabel("Execution time (s)", color=color)
    ax2.plot(execution_times, color=color)
    ax2.tick_params(axis="y", labelcolor=color)
    ax2.yaxis.get_major_formatter().set_useOffset(False)

    fig.tight_layout()
    plt.title("GPU Memory Consumption and Execution Time for Each Run")
    fig.subplots_adjust(top=0.88)
    plt.savefig(f"{dir}/falcon_memory_time.png")

model_path = "tiiuae/falcon-40b-instruct"
config = AutoConfig.from_pretrained(model_path, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(model_path, config=config)
tokenizer = AutoTokenizer.from_pretrained(model_path)

runs = 5
mem_consumptions = []
execution_times = []

for i in range(runs):
    prompts = [
    "Explain the question",
    "Suggest me the optimal Approach",
    "Tell me a hint",
    "What is optimal Time Complexity",
    "Be My Interviewer",
    ]

    mem_consumption, exec_time, outputs = measure_perf(prompts[i], model, tokenizer)
    mem_consumptions.append(mem_consumption)
    execution_times.append(exec_time)
    print(tokenizer.decode(outputs[0]))

plot_results(mem_consumptions, execution_times)

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py:3553 in run_code        │
│                                                                                                  │
│   3550 │   │   │   │   elif async_ :                                                             │
│   3551 │   │   │   │   │   await eval(code_obj, self.user_global_ns, self.user_ns)               │
│   3552 │   │   │   │   else:                                                                     │
│ ❱ 3553 │   │   │   │   │   exec(code_obj, self.user_global_ns, self.user_ns)                     │
│   3554 │   │   │   finally:                                                                      │
│   3555 │   │   │   │   # Reset our crash handler in place                                        │
│   3556 │   │   │   │   sys.excepthook = old_excepthook                                           │
│ in <cell line: 9>:9                                                                              │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/transformers/pipelines/__init__.py:788 in pipeline       │
│                                                                                                  │
│   785 │   # Forced if framework already defined, inferred if it's None                           │
│   786 │   # Will load the correct model if possible                                              │
│   787 │   model_classes = {"tf": targeted_task["tf"], "pt": targeted_task["pt"]}                 │
│ ❱ 788 │   framework, model = infer_framework_load_model(                                         │
│   789 │   │   model,                                                                             │
│   790 │   │   model_classes=model_classes,                                                       │
│   791 │   │   config=config,                                                                     │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:269 in                    │
│ infer_framework_load_model                                                                       │
│                                                                                                  │
│    266 │   │   │   │   )                                                                         │
│    267 │   │   │                                                                                 │
│    268 │   │   │   try:                                                                          │
│ ❱  269 │   │   │   │   model = model_class.from_pretrained(model, **kwargs)                      │
│    270 │   │   │   │   if hasattr(model, "eval"):                                                │
│    271 │   │   │   │   │   model = model.eval()                                                  │
│    272 │   │   │   │   # Stop loading on the first successful load.                              │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/transformers/models/auto/auto_factory.py:479 in          │
│ from_pretrained                                                                                  │
│                                                                                                  │
│   476 │   │   │   │   class_ref, pretrained_model_name_or_path, **hub_kwargs, **kwargs           │
│   477 │   │   │   )                                                                              │
│   478 │   │   │   _ = hub_kwargs.pop("code_revision", None)                                      │
│ ❱ 479 │   │   │   return model_class.from_pretrained(      